# First Model Iteration - Healthy vs. Pneumonia 

### Data Pre-Processing

In [2]:
#all necessary imports
import time
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split

from sklearn import svm
from sklearn.model_selection import GridSearchCV

import xray_data #y data: 1 = NORMAL, 0 = PNEUMONIA

In [4]:
#Constants
scale = 200
#only load in the normal and pneumonia values
label_filter = ['NORMAL','PNEUMONIA']
subset = 'PROP' # either "EQL" or "PROP"
random.seed(207)

In [3]:
#load in dev data
X_dev_orig, y_dev_orig = xray_data.load_val(scale,label_filter,subset=subset)
print(f'X_dev_orig, y_dev_orig shape: {X_dev_orig.shape, y_dev_orig.shape}')
print(f'y_dev_orig shape for NORMAL cases: {y_dev_orig[y_dev_orig ==1].shape}')
print('----')

# cut of for training samples of each class, only 230 normal rows
test_cutoff = 230

#load in test data - won't be used until very end
X_test, y_test = xray_data.load_test(scale,label_filter,test_cutoff,subset=subset)
print(f'X_test, y_test shape: {X_test.shape, y_test.shape}')
print(f'y_test shape for NORMAL cases: {y_test[y_test ==1].shape}')
print('----')

# cut of for training samples of each class, only 1300 normal rows
train_cutoff = 1300

X_train, y_train = xray_data.load_train(scale,label_filter,subset=subset)

100% |#########################################################################|
100% |#########################################################################|
100% |#########################################################################|
100% |#########################################################################|
100% |#########################################################################|


PNEUMONIA: 8
NORMAL: 8
Total: 16
X_dev_orig, y_dev_orig shape: ((16, 40000), (16,))
y_dev_orig shape for NORMAL cases: (8,)
----


100% |#########################################################################|
100% |#########################################################################|
100% |#########################################################################|
100% |#########################################################################|


PNEUMONIA: 143
NORMAL: 86
Total: 229
X_test, y_test shape: ((229, 40000), (229,))
y_test shape for NORMAL cases: (86,)
----


100% |#########################################################################|
100% |#########################################################################|
100% |#########################################################################|


PNEUMONIA: 3875
NORMAL: 1341
Total: 5216


In [4]:
#normalize image using functions from xray_data file
(mean,std) = xray_data.find_mean_std(X_train)
print(mean,std)
print(X_train.shape)
adjusted_X_train = xray_data.normalize_images(X_train,mean,std)
print (f'adjusted train {adjusted_X_train.shape}')
adjusted_X_dev_org = xray_data.normalize_images(X_dev_orig,mean,std)
adjusted_X_test = xray_data.normalize_images(X_test,mean,std)

0.4799373263138842 0.23625422461763224
(5216, 40000)
adjusted train (5216, 40000)


In [5]:
#split the train set into train and dev for intermediate testing
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size = .1, stratify = y_train )
print(f'X_train, y_train shape: {X_train.shape, y_train.shape}')
print(f'y_train shape for NORMAL cases: {y_train[y_train ==1].shape}')
print('----')
print(f'X_dev, y_dev shape: {X_dev.shape, y_dev.shape}')
print(f'y_dev shape for NORMAL cases: {y_dev[y_dev ==1].shape}')

X_train, y_train shape: ((4694, 40000), (4694,))
y_train shape for NORMAL cases: (1207,)
----
X_dev, y_dev shape: ((522, 40000), (522,))
y_dev shape for NORMAL cases: (134,)


### Model 1A: Single Layer Perceptron

In [7]:
#define model, fit on train, predict on dev, print accuracy 
per_mod = Perceptron(random_state = 42)
per_mod.fit(X_train, y_train)
per_pred = per_mod.predict(X_dev)
print(f'Single Layer Perceptron Model Accuracy: {accuracy_score(per_pred,y_dev)*100:9.5}')

Single Layer Perceptron Model Accuracy:    96.552


### Model 1B: Multi-Layer Perceptron

In [11]:
#use default parameters for MLP model
mlp_mod = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5,2), max_iter = 1000, random_state=1)
mlp_mod.fit(X_train, y_train)
mlp_pred = mlp_mod.predict(X_dev)
print(f'Multi Layer Perceptron Model Accuracy: {accuracy_score(mlp_pred,y_dev)*100:9.5}')

Multi Layer Perceptron Model Accuracy:    95.211


Because for a single model implementation we got a higher accuracy using the multi-layer perceptron, that's the model we will run our GridSearchCV on.

### Model 1C: Multi-Layer Perceptron with GridsearchCV

In [18]:
#conduct a gridsearch over the following parameters 
#this also does 5 fold cross validation to generate "best_score"
#which helps us decide on the optimal parameters
parameters = {'solver': ['lbfgs'],
              'activation': ['identity', 'logistic'],
              'hidden_layer_sizes': [(5,2), (5,)],
              'alpha': [10, 1, 0.1, 0.0001],
              'max_iter': [1000]
             }
clf = GridSearchCV(MLPClassifier(), parameters,verbose=3)
clf.fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END activation=identity, alpha=10, hidden_layer_sizes=(5, 2), max_iter=1000, solver=lbfgs;, score=0.963 total time= 1.3min
[CV 2/5] END activation=identity, alpha=10, hidden_layer_sizes=(5, 2), max_iter=1000, solver=lbfgs;, score=0.968 total time= 1.2min
[CV 3/5] END activation=identity, alpha=10, hidden_layer_sizes=(5, 2), max_iter=1000, solver=lbfgs;, score=0.962 total time= 1.2min
[CV 4/5] END activation=identity, alpha=10, hidden_layer_sizes=(5, 2), max_iter=1000, solver=lbfgs;, score=0.969 total time= 1.0min
[CV 5/5] END activation=identity, alpha=10, hidden_layer_sizes=(5, 2), max_iter=1000, solver=lbfgs;, score=0.968 total time= 1.1min
[CV 1/5] END activation=identity, alpha=10, hidden_layer_sizes=(5,), max_iter=1000, solver=lbfgs;, score=0.963 total time= 1.1min
[CV 2/5] END activation=identity, alpha=10, hidden_layer_sizes=(5,), max_iter=1000, solver=lbfgs;, score=0.969 total time=  52.3s
[CV 3/5] END activa

/Users/alexandradrossos/miniforge3/envs/tf_m1/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV 5/5] END activation=logistic, alpha=10, hidden_layer_sizes=(5,), max_iter=1000, solver=lbfgs;, score=0.971 total time= 3.1min
[CV 1/5] END activation=logistic, alpha=1, hidden_layer_sizes=(5, 2), max_iter=1000, solver=lbfgs;, score=0.743 total time=   6.0s
[CV 2/5] END activation=logistic, alpha=1, hidden_layer_sizes=(5, 2), max_iter=1000, solver=lbfgs;, score=0.743 total time=   4.0s
[CV 3/5] END activation=logistic, alpha=1, hidden_layer_sizes=(5, 2), max_iter=1000, solver=lbfgs;, score=0.742 total time=   3.7s
[CV 4/5] END activation=logistic, alpha=1, hidden_layer_sizes=(5, 2), max_iter=1000, solver=lbfgs;, score=0.742 total time=   3.1s
[CV 5/5] END activation=logistic, alpha=1, hidden_layer_sizes=(5, 2), max_iter=1000, solver=lbfgs;, score=0.743 total time=   3.3s


/Users/alexandradrossos/miniforge3/envs/tf_m1/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV 1/5] END activation=logistic, alpha=1, hidden_layer_sizes=(5,), max_iter=1000, solver=lbfgs;, score=0.963 total time= 3.1min
[CV 2/5] END activation=logistic, alpha=1, hidden_layer_sizes=(5,), max_iter=1000, solver=lbfgs;, score=0.743 total time=   6.7s
[CV 3/5] END activation=logistic, alpha=1, hidden_layer_sizes=(5,), max_iter=1000, solver=lbfgs;, score=0.963 total time= 1.4min
[CV 4/5] END activation=logistic, alpha=1, hidden_layer_sizes=(5,), max_iter=1000, solver=lbfgs;, score=0.966 total time= 2.9min
[CV 5/5] END activation=logistic, alpha=1, hidden_layer_sizes=(5,), max_iter=1000, solver=lbfgs;, score=0.967 total time= 1.4min
[CV 1/5] END activation=logistic, alpha=0.1, hidden_layer_sizes=(5, 2), max_iter=1000, solver=lbfgs;, score=0.743 total time=   2.3s
[CV 2/5] END activation=logistic, alpha=0.1, hidden_layer_sizes=(5, 2), max_iter=1000, solver=lbfgs;, score=0.743 total time=   2.2s
[CV 3/5] END activation=logistic, alpha=0.1, hidden_layer_sizes=(5, 2), max_iter=1000, so

GridSearchCV(estimator=MLPClassifier(),
             param_grid={'activation': ['identity', 'logistic'],
                         'alpha': [10, 1, 0.1, 0.0001],
                         'hidden_layer_sizes': [(5, 2), (5,)],
                         'max_iter': [1000], 'solver': ['lbfgs']},
             verbose=3)

In [20]:
print(f'Best score: {clf.best_score_*100:9.5}')
print(f'Best params: {clf.best_params_}')

Best score:    96.677
Best params: {'activation': 'identity', 'alpha': 1, 'hidden_layer_sizes': (5, 2), 'max_iter': 1000, 'solver': 'lbfgs'}


In [21]:
#predict on dev using the optimal model
cv_mlp_pred = clf.predict(X_dev)
print(f'Best Fit Model accuracy with Lbfgs: {accuracy_score(cv_mlp_pred,y_dev)*100:9.5}')

Best Fit Model accuracy with Lbfgs:    95.977


# Second Model Iteration - Healthy vs. Pneumonia vs. COVID vs. TB

### Data Pre-Processing

In [4]:
#define constants needed for loading in data
scale = 200
label_filter = ['NORMAL','PNEUMONIA','COVID19','TURBERCULOSIS']
subset = 'PROP' # either "EQL" or "PROP"
random.seed(207)

In [5]:
#load in train data
#0=Normal, 1=Pn, 2=COVID, 3=TB
X_train_all, y_train_all = xray_data.load_train(scale,label_filter, subset=subset)
print(X_train_all.shape)
print(y_train_all.shape)

100% |#########################################################################|
100% |#########################################################################|
100% |#########################################################################|
100% |#########################################################################|


COVID19: 460
PNEUMONIA: 3875
NORMAL: 1341
TURBERCULOSIS: 650
Total: 6326
(6326, 40000)
(6326,)


In [6]:
#split the train dataset into train and dev 80-20 split
X_mini_train_all, X_dev_all, y_mini_train_all, y_dev_all = train_test_split(X_train_all, y_train_all, test_size = .2, stratify = y_train_all)
print(X_mini_train_all.shape)
print(X_dev_all.shape)

(5060, 40000)
(1266, 40000)


In [21]:
print(f'Train Set Value Counts:')
print(f'COVID19: {len(y_mini_train_all[y_mini_train_all ==2])}')
print(f'PNEUMONIA: {len(y_mini_train_all[y_mini_train_all ==1])}')
print(f'NORMAL: {len(y_mini_train_all[y_mini_train_all ==0])}')
print(f'TURBERCULOSIS: {len(y_mini_train_all[y_mini_train_all ==3])}')

Mini Train Value Counts:
COVID19: 368
PNEUMONIA: 3099
NORMAL: 1073
TURBERCULOSIS: 520


In [22]:
print(f'Dev Set Value Counts:')
print(f'COVID19: {len(y_dev_all[y_dev_all==2])}')
print(f'PNEUMONIA: {len(y_dev_all[y_dev_all ==1])}')
print(f'NORMAL: {len(y_dev_all[y_dev_all ==0])}')
print(f'TURBERCULOSIS: {len(y_dev_all[y_dev_all ==3])}')

Dev Set Value Counts:
COVID19: 92
PNEUMONIA: 776
NORMAL: 268
TURBERCULOSIS: 130


In [24]:
X_test, y_test = xray_data.load_test(scale,label_filter,subset=subset)

100% |#########################################################################|
100% |#########################################################################|
100% |#########################################################################|
100% |#########################################################################|


COVID19: 106
PNEUMONIA: 390
NORMAL: 234
TURBERCULOSIS: 41
Total: 771


In [25]:
print(f'Test Set Value Counts:')
print(f'COVID19: {len(y_test[y_test==2])}')
print(f'PNEUMONIA: {len(y_test[y_test ==1])}')
print(f'NORMAL: {len(y_test[y_test ==0])}')
print(f'TURBERCULOSIS: {len(y_test[y_test ==3])}')

Test Set Value Counts:
COVID19: 106
PNEUMONIA: 390
NORMAL: 234
TURBERCULOSIS: 41


### Model 2A: Single Layer Perceptron

In [10]:
per_mod = Perceptron(random_state = 46)
per_mod.fit(X_train_all, y_train_all)
per_pred = per_mod.predict(X_dev_all)
print(f'Single Layer Perceptron Model Accuracy: {accuracy_score(per_pred,y_dev_all)*100:9.5}')

Single Layer Perceptron Model Accuracy:    94.471


### Model 2B: Default Multi Layer Perceptron

In [11]:
mlp_mod = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5,2), max_iter = 1000, random_state=1)
mlp_mod.fit(X_train_all, y_train_all)
mlp_pred = mlp_mod.predict(X_dev_all)
print(f'Multi Layer Perceptron Model Accuracy: {accuracy_score(mlp_pred,y_dev_all)*100:9.5}')

Multi Layer Perceptron Model Accuracy:    99.842


### Model 2C: Cross Validated GridSearch MLP

In [24]:
parameters = {'solver': ['lbfgs'],
              'activation': ['identity', 'logistic'],
              'hidden_layer_sizes': [(5,)],
              'alpha': [10, 1, 0.1],
              'max_iter': [5000]
             }
clf = GridSearchCV(MLPClassifier(), parameters,verbose = 3)
clf.fit(X_train_all,y_train_all)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END activation=identity, alpha=10, hidden_layer_sizes=(5,), max_iter=5000, solver=lbfgs;, score=0.938 total time= 4.8min
[CV 2/5] END activation=identity, alpha=10, hidden_layer_sizes=(5,), max_iter=5000, solver=lbfgs;, score=0.938 total time= 4.9min
[CV 3/5] END activation=identity, alpha=10, hidden_layer_sizes=(5,), max_iter=5000, solver=lbfgs;, score=0.919 total time= 3.9min
[CV 4/5] END activation=identity, alpha=10, hidden_layer_sizes=(5,), max_iter=5000, solver=lbfgs;, score=0.932 total time= 4.6min
[CV 5/5] END activation=identity, alpha=10, hidden_layer_sizes=(5,), max_iter=5000, solver=lbfgs;, score=0.939 total time= 5.1min
[CV 1/5] END activation=identity, alpha=1, hidden_layer_sizes=(5,), max_iter=5000, solver=lbfgs;, score=0.937 total time= 2.0min
[CV 2/5] END activation=identity, alpha=1, hidden_layer_sizes=(5,), max_iter=5000, solver=lbfgs;, score=0.936 total time= 2.0min
[CV 3/5] END activation=identity

GridSearchCV(estimator=MLPClassifier(),
             param_grid={'activation': ['identity', 'logistic'],
                         'alpha': [10, 1, 0.1], 'hidden_layer_sizes': [(5,)],
                         'max_iter': [5000], 'solver': ['lbfgs']},
             verbose=3)

In [25]:
print(f'Best score: {clf.best_score_*100:9.5}')
print(f'Best params: {clf.best_params_}')

Best score:    93.598
Best params: {'activation': 'logistic', 'alpha': 10, 'hidden_layer_sizes': (5,), 'max_iter': 5000, 'solver': 'lbfgs'}


In [29]:
cv_mlp_pred = clf.predict(X_dev_all)
print(f'Model Accuracy: {accuracy_score(cv_mlp_pred,y_dev_all)*100:9.5}')

Model Accuracy:     100.0


Best params: {'activation': 'logistic', 'alpha': 10, 'hidden_layer_sizes': (5,), 'max_iter': 5000, 'solver': 'lbfgs'}

In [7]:
best_mlp = MLPClassifier(solver='lbfgs', alpha=10, hidden_layer_sizes=(5,), max_iter = 5000, 
                         activation = 'logistic',random_state=1)

best_mlp.fit(X_train_all, y_train_all)
best_mlp_pred = best_mlp.predict(X_test)
print(f'Accuracy of best MLP on test data: {accuracy_score(best_mlp_pred,y_test)*100:9.5}')

Accuracy of best MLP on test data:    75.357
